In [1]:
%load_ext autoreload
%autoreload 2
from types import SimpleNamespace
import json
import os
import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from models.efficient_unet import AbstractUNet
from dataset import CatDataset
from inpaint_tools import read_file_list
from skimage import io
import numpy as np

c:\Users\lowes\anaconda3\envs\DeepLearning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args = {"unet": {"block": "ffmmm", #m=MBConv,f=FusedMBConv,u=Unet 
                    "act": "silu",
                    "res_mode": "cat", #cat, add
                    "init_mode": "effecientnetv2",
                    "downscale_mode": "avgpool",
                    "upscale_mode": "bilinear",
                    "input_channels": 4,
                    "output_channels": 3,
                    "num_blocks": 5,
                    "num_c": [8,16,32,48,64],
                    "num_repeat": [1,2,2,4,4],
                    "expand_ratio": [1,4,4,6,6],
                    "SE": [0,0,1,1,1]
                }}

args = {"unet": {"block": "ffmmm", #m=MBConv,f=FusedMBConv,u=Unet 
                    "act": "silu",
                    "res_mode": "cat", #cat, add
                    "init_mode": "effecientnetv2",
                    "downscale_mode": "avgpool",
                    "upscale_mode": "bilinear",
                    "input_channels": 4,
                    "output_channels": 3,
                    "num_blocks": 4,
                    "num_c": [8,16,32,64],
                    "num_repeat": [1,2,2,4],
                    "expand_ratio": [1,4,4,6],
                    "SE": [0,0,1,1]
                }}
args_n = json.loads(json.dumps(args), object_hook=lambda item: SimpleNamespace(**item))

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
net = AbstractUNet(args_n).to(device)

In [4]:
test = torch.zeros((1,4,360,360)).to(device)
output = net(test)
output.shape

torch.Size([1, 3, 360, 360])

In [6]:
settings = {
	"team_data":
	{
		"name": "YourTeamNameHere"
	},
    "training_parms": {
		"method": "YourMethodHere",
		"dummy_value": 1000
    },
    "dirs": {
            "input_data_dir": "C:/Users/lowes/OneDrive/Skrivebord/DTU/summer_school_23/MissingDataChallenge/data/",
			"output_data_dir": "missing_data_output/"
	},
	"challenge_server": {
		"address": "http://fungi.compute.dtu.dk:8080"
	},
	"data_set": "training",
	"batch_size": 2,
	"num_workers": 0,
}


dataset_train = CatDataset(settings)
print([d.shape for d in dataset_train[0]])

dl = DataLoader(dataset_train, 
				batch_size=settings["batch_size"],
                shuffle=True,
                num_workers=settings["num_workers"])

[torch.Size([4, 360, 360]), torch.Size([1, 360, 360]), torch.Size([3, 360, 360])]


## Train

In [9]:
dataset_train = CatDataset(settings)
dl_train = DataLoader(dataset_train, 
				batch_size=settings["batch_size"],
                shuffle=True,
                num_workers=settings["num_workers"])

settings["data_set"] = "validation_200"
dataset_val = CatDataset(settings)
dl_val = DataLoader(dataset_val, 
				batch_size=settings["batch_size"],
                shuffle=True,
                num_workers=settings["num_workers"])

# Create the model, loss function, and optimizer
NUM_EPOCHS = 12

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AbstractUNet(args_n).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0

    # Create a progress bar using tqdm
    with tqdm.tqdm(dl_train, unit="batch") as tepoch:
        for batch in tepoch:
            model_input, mask, image = batch
            model_input, mask, image = model_input.to(device), mask.to(device), image.to(device)
            
            optimizer.zero_grad()

            # Forward pass
            outputs = model(model_input)

            loss = criterion(outputs*mask, image*mask)

            # Backpropagation and optimization
            loss.backward()
            optimizer.step()

            # Update progress bar description with loss
            tepoch.set_description(f"Epoch {epoch+1}")
            tepoch.set_postfix({"loss"=loss.item()})

            # Accumulate loss for this epoch
            running_loss += loss.item()

    # Calculate and print average loss for the epoch
    average_loss = running_loss / len(train_loader)

    for batch in dl_val:
        model_input, mask, image = batch
        model_input, mask, image = model_input.to(device), mask.to(device), image.to(device)


Epoch 1:   3%|▎         | 66/2484 [00:10<06:35,  6.11batch/s, loss=2.88e+4]


KeyboardInterrupt: 